In [1]:
!nvidia-smi

Wed Jul  7 17:12:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 471.11       CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| N/A   68C    P8     6W /  N/A |   2194MiB /  6144MiB |    ERR!      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
import os
import pandas as pd

import sys
sys.path.append('..')

from deepmatcher.data.custom_dataset import DeepMatcherDataset
from deepmatcher.data.encoder.text_encoders import HFTextEncoder
# from deepmatcher.data.encoder.image_encoders import DINOImageEncoder
from deepmatcher.optim import Optimizer

from torch.utils.data import DataLoader
from deepmatcher.models.core import MatchingModel

In [2]:
dataset_dir = "sample_data/itunes-amazon/"
train_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('train:', train_df.shape)
val_df = pd.read_csv(os.path.join(dataset_dir, 'validation.csv'))
print('val:', val_df.shape)
test_df = pd.read_csv(os.path.join(dataset_dir, 'train.csv'))
print('test:', test_df.shape)

# for df in (train_df, val_df, test_df):
#     df['left_image'] = df['right_image'] = os.path.join(dataset_dir, 'test_img.jpg')
train_df.head()

train: (323, 18)
val: (108, 18)
test: (323, 18)


,id,label,left_Song_Name,left_Artist_Name,left_Album_Name,left_Genre,left_Price,left_CopyRight,left_Time,left_Released,right_Song_Name,right_Artist_Name,right_Album_Name,right_Genre,right_Price,right_CopyRight,right_Time,right_Released
0,448,0,Baby When the Light ( David Guetta & Fred Rist...,David Guetta,Pop Life ( Extended Version ) [ Bonus Version ],"Dance , Music , Rock , Pop , House , Electroni...",$ 1.29,‰ ãÑ 2007 Gum Records,6:17,18-Sep-07,Revolver ( Madonna Vs. David Guetta Feat . Lil...,David Guetta,One Love ( Deluxe Version ),Dance & Electronic,$ 1.29,( C ) 2014 Swedish House Mafia Holdings Ltd ( ...,3:18,"August 21 , 2009"
1,287,1,Outversion,Mark Ronson,Version,"Pop , Music , R&B / Soul,Soul,Dance,Rock,Jazz,...",$ 0.99,2007 Mark Ronson under exclusive license to SO...,1:50,10-Jul-07,Outversion,Mark Ronson,Version [ Explicit ],Pop,$ 0.99,( c ) 2011 J'adore Records,1:50,"July 10 , 2007"
2,534,0,Peer Pressure ( feat . Traci Nelson ),Snoop Dogg,Doggumentary,"Hip-Hop/Rap , Music , Rock , Gangsta Rap , Wes...",$ 1.29,"‰ ãÑ 2011 Capitol Records , LLC . All rights r...",4:07,29-Mar-11,Boom ( ( Feat . T-Pain ) [ Edited ] ),Snoop Dogg,Doggumentary [ Edited ],"Rap & Hip-Hop , West Coast",$ 1.29,"( C ) 2011 Capitol Records , LLC",3:50,"March 29 , 2011"
3,181,1,Stars Come Out ( Tim Mason Remix ),Zedd,Stars Come Out ( Remixes ) - EP,"Dance , Music , Electronic , House",$ 1.29,2012 Dim Mak Inc.,5:49,20-May-14,Stars Come Out ( Dillon Francis Remix ),Zedd,Stars Come Out [ Dillon Francis Remix ],Dance & Electronic,$ 1.29,2012 Dim Mak Inc.,4:08,"May 20 , 2014"
4,485,0,Jump ( feat . Nelly Furtado ),Flo Rida,R.O.O.T.S. ( Deluxe Version ),"Hip-Hop/Rap , Music",$ 1.29,‰ ãÑ 2009 Atlantic Recording Corporation for t...,3:28,30-Mar-09,"Yayo [ Feat . Brisco , Billy Blue , Ball Greez...",Flo Rida,R.O.O.T.S. ( Route Of Overcoming The Struggle ...,Rap & Hip-Hop,$ 1.29,"( C ) 2012 Motown Records , a Division of UMG ...",7:53,"March 30 , 2009"


In [3]:
text_encoder = HFTextEncoder('sentence-transformers/paraphrase-albert-small-v2', trainable=False, seqtovec='mean')
# image_encoder = DINOImageEncoder(device='cuda')


In [4]:
label_column = 'label'
text_columns = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
# image_col = 'image'
image_col = ''
tokenizer = text_encoder.tokenizer

def get_dataset(data_df, label_column, text_columns, image_col, tokenizer):
    return DeepMatcherDataset(
        data_df=data_df,
        label_col=label_column,
        text_cols=text_columns,
        image_col=image_col,
        tokenizer=tokenizer
        )

def get_dataloader(dataset, batch_size, shuffle, num_workers):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, collate_fn=dataset.collate_fn)

batch_size = 32
num_workers = 0

train_dataset = get_dataset(train_df, label_column, text_columns, image_col, tokenizer)
val_dataset = get_dataset(val_df, label_column, text_columns, image_col, tokenizer)
test_dataset = get_dataset(test_df, label_column, text_columns, image_col, tokenizer)

train_dataloader = get_dataloader(train_dataset, batch_size, True, num_workers)
val_dataloader = get_dataloader(val_dataset, batch_size, False, num_workers)
test_dataloader = get_dataloader(test_dataset, batch_size, False, num_workers)

received data_df : (323, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label
received data_df : (108, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label
received data_df : (323, 18)
supplied text columns : Song_Name, Artist_Name, Album_Name, Genre, Price, CopyRight, Time, Released
supplied image column : 
supplied label column : label


In [5]:
model = MatchingModel(
    text_encoder=text_encoder,
    # image_encoder=image_encoder,
    attr_summarizer=None,
    attr_condense_factor='auto',
    attr_comparator='concat',
    attr_merge='concat',
    classifier='2-layer-highway',
    hidden_size=300,
    text_attrs=text_columns,
    # image_attr='image'
    )

text_attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
image_attr: 
prefixes: ('left_', 'right_')
attrs: ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']
all attrs: ['left_Song_Name', 'right_Song_Name', 'left_Artist_Name', 'right_Artist_Name', 'left_Album_Name', 'right_Album_Name', 'left_Genre', 'right_Genre', 'left_Price', 'right_Price', 'left_CopyRight', 'right_CopyRight', 'left_Time', 'right_Time', 'left_Released', 'right_Released']


In [6]:
next(iter(train_dataloader))

{'attrs': {'Song_Name': {'left_': {'input_ids': tensor([[    2,    20,    93,  ...,     0,     0,     0],
            [    2,  1632,   130,  ...,     0,     0,     0],
            [    2,   246,    16,  ...,     0,     0,     0],
            ...,
            [    2, 20797,    70,  ...,     0,     0,     0],
            [    2,    55,  4673,  ...,     0,     0,     0],
            [    2,  1518,    42,  ...,     0,     0,     0]]),
    'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            ...,
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0],
            [0, 0, 0,  ..., 0, 0, 0]]),
    'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            ...,
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0],
            [1, 1, 1,  ..., 0, 0, 0]])},
   'right_': {'input_ids': tensor(

In [15]:
model.initialize(next(iter(train_dataloader))['attrs'])
model

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument index in method wrapper_index_select)

In [16]:
optimizer = Optimizer(lr=1e-3, lr_decay=0.5, start_decay_at=3)

In [7]:
model.run_train(
    train_dataset,
    val_dataset,
    best_save_path='best_mode.pth',
    epochs=20,
    batch_size=batch_size
    )

D:\installations\anaconda\envs\aisle3-torch\lib\site-packages\torch\nn\modules\module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 1277002
===>  TRAIN Epoch 1


D:\installations\anaconda\envs\aisle3-torch\lib\site-packages\torch\nn\functional.py:2741: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(
0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:   12.9 | Load Time:    0.7 || F1:  14.89 | Prec:  41.18 | Rec:   9.09 || Ex/s:  23.78

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    2.9 | Load Time:    0.1 || F1:  52.94 | Prec:  90.00 | Rec:  37.50 || Ex/s:  36.37

Updating Learning Rate: 8.000e-04 Acc: 52.94117736816406 Epoch: 1
* Best F1: tensor(52.9412, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


...

0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 12 || Run Time:    8.8 | Load Time:    0.2 || F1:  99.35 | Prec:  98.72 | Rec: 100.00 || Ex/s:  35.77

===>  EVAL Epoch 12
Finished Epoch 12 || Run Time:    2.7 | Load Time:    0.1 || F1:  69.57 | Prec:  72.73 | Rec:  66.67 || Ex/s:  38.20

Updating Learning Rate: 6.872e-05 Acc: 69.5652084350586 Epoch: 12
---------------------

===>  TRAIN Epoch 13


0% [██] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 13 || Run Time:    9.1 | Load Time:    0.2 || F1:  99.35 | Prec:  98.72 | Rec: 100.00 || Ex/s:  34.49

===>  EVAL Epoch 13
Finished Epoch 13 || Run Time:    2.7 | Load Time:    0.1 || F1:  69.57 | Prec:  72.73 | Rec:  66.67 || Ex/s:  38.00

Updating Learning Rate: 5.498e-05 Acc: 69.5652084350586 Epoch: 13
---------------------

===>  TRAIN Epoch 14


KeyboardInterrupt: 

In [ ]:
model.run_eval(test_dataset)